In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

# Initialize SageMaker client and session
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name

# Set bucket name for storing data in S3
bucket = 'ccsuncleanv2'
print("Using bucket " + bucket)

# https://www.youtube.com/watch?v=Le-A72NjaWs

ImportError: _multiarray_umath failed to import

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/reubensantoso/Library/Application Support/sagemaker/config.yaml


RuntimeError: CPU dispatcher tracer already initlized

Using bucket ccsuncleanv2


In [2]:
# Testing S3 connection
# Initialize the S3 client

s3 = boto3.client("s3")
try:
    # List objects in the specified bucket
    response = s3.list_objects_v2(Bucket=bucket)
    
    # Check if the bucket is accessible
    if 'Contents' in response:
        print("Connection successful. Files in bucket:")
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("Connection successful, but the bucket is empty.")
except Exception as e:
    print(f"Connection failed: {e}")


Connection successful. Files in bucket:
sklearncontainer/x-test.csv
sklearncontainer/x-train.csv
sklearncontainer/y-test.csv
sklearncontainer/y-train.csv


In [3]:
import sqlite3

# Use the path to your database file
db_path = "C3S_V2.db"

try:
    # Connect to the database
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    
    # Check if the master table exists by running a simple query
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cur.fetchall()
    if ("master",) in tables:
        print("Master table exists and connection is successful.")
    else:
        print("Master table not found.")
        
    # Optionally, print all tables to verify
    print("Tables in database:", tables)

except sqlite3.OperationalError as e:
    print(f"OperationalError: {e}")
finally:
    con.close()


Master table exists and connection is successful.
Tables in database: [('master',), ('mqns',), ('fingerprints',)]


In [4]:
from c3sdb.ml.data import C3SD

data = C3SD("C3S_V2.db", seed=2345)
data.assemble_features(encoded_adduct=True, mqn_indices="all", handle_nan="impute")  # Or handle_nan="drop"

X_train, X_test, Y_train, Y_test = data.train_test_split("ccs")


In [5]:
import pandas as pd

# Assuming X_train, X_test, Y_train, and Y_test are pandas DataFrames or arrays
# Convert to DataFrame if they are numpy arrays for ease of sampling
# X_train_sample = pd.DataFrame(X_train).sample(n=1000, random_state=42)
# X_test_sample = pd.DataFrame(X_test).sample(n=1000, random_state=42)
# Y_train_sample = pd.DataFrame(Y_train).sample(n=1000, random_state=42)
# Y_test_sample = pd.DataFrame(Y_test).sample(n=1000, random_state=42)

X_train_sample = pd.DataFrame(X_train)
X_test_sample = pd.DataFrame(X_test)
Y_train_sample = pd.DataFrame(Y_train)
Y_test_sample = pd.DataFrame(Y_test)

# Now you have 1000 random samples from each dataset
print("Sampled X_train:", X_train_sample)
print("Sampled X_test:", X_test_sample)
print("Sampled Y_train:", Y_train_sample)
print("Sampled Y_test:", Y_test_sample)


Sampled X_train:                0    1    2    3    4    5    6    7    8    9   ...   44   45   
0      210.077500  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  \
1      277.086900  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2      293.042600  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0   
3      411.264700  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0   
4      482.133826  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
15131  355.214200  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
15132  275.091900  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
15133  276.063200  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
15134  178.254300  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
15135  497.287381  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

        46

In [6]:
X_train_sample.to_csv("x-train.csv")
X_test_sample.to_csv("x-test.csv")
Y_train_sample.to_csv("y-train.csv")
Y_test_sample.to_csv("y-test.csv")

In [7]:
#uploading to s3 bucket
sk_prefix = "sklearncontainer"

trainpath_x = sess.upload_data(
    path="x-train.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath_x = sess.upload_data(
    path="x-test.csv", bucket=bucket, key_prefix=sk_prefix
)

trainpath_y = sess.upload_data(
    path="y-train.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath_y = sess.upload_data(
    path="y-test.csv", bucket=bucket, key_prefix=sk_prefix
)

In [8]:
print(f"X_train shape: {X_train_sample}") #1000 x 54 columns
print(f"y_train shape: {Y_train_sample}") #1000 x 1 column
print(f"X_test shape: {X_test_sample}") #1000 x 54 columns
print(f"y_test shape: {Y_test_sample}") #1000 x 1 columns


X_train shape:                0    1    2    3    4    5    6    7    8    9   ...   44   45   
0      210.077500  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  \
1      277.086900  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2      293.042600  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0   
3      411.264700  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0   
4      482.133826  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
15131  355.214200  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
15132  275.091900  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
15133  276.063200  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
15134  178.254300  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
15135  497.287381  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

        46  

In [27]:
from sagemaker.sklearn.estimator import SKLearn

sklearn_estimator = SKLearn(
    entry_point="sagemaker_train.py",
    # source_dir=".",  # Ensure this is the correct relative path
    role="arn:aws:iam::715841358910:role/service-role/AmazonSageMaker-ExecutionRole-20241029T013495",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="0.23-1",  # Confirm that this version supports Conda
    hyperparameters={
        "n_estimators": 100,
        "max_depth": 5
    },
    use_spot_instances=True,
    max_wait=3600,
    max_run=1800,
    # dependencies=["environment.yaml"],  # Include the environment file
    # py_version="py3",  # Ensure you're using Python 3
)

In [28]:
sklearn_estimator.fit({
    'trainx': trainpath_x,
    'trainy': trainpath_y,
    'testx': testpath_x,
    'testy': testpath_y,
})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-11-03-06-56-49-999


2024-11-03 06:56:52 Starting - Starting the training job...
2024-11-03 06:57:06 Starting - Preparing the instances for training...
2024-11-03 06:57:30 Downloading - Downloading input data...
2024-11-03 06:58:00 Downloading - Downloading the training image...
2024-11-03 06:58:47 Training - Training image download completed. Training in progress.
2024-11-03 06:58:47 Uploading - Uploading generated training model2024-11-03 06:58:41,151 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-03 06:58:41,155 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-03 06:58:41,192 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-03 06:58:41,388 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-03 06:58:41,400 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-03 06:58:41,411 sagemaker-training-toolkit INFO

UnexpectedStatusException: Error for Training job sagemaker-scikit-learn-2024-11-03-06-56-49-999: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 100, in run
    wait, capture_error
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 291, in run
    cwd=environment.code_dir,
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 208, in check_error
    info=extra_info,
sagemaker_training.errors.E